In [2]:

import pandas as pd

In [3]:
from newsapi import NewsApiClient

In [4]:
newsapi = NewsApiClient(api_key='30449eaef55c4cf195a595e35920efc0')


In [5]:
# /v2/top-headlines
headlines = newsapi.get_top_headlines(q='AI',
                                        category='health',
                                          language='en',
                                          country='us')

headlines['articles']

[{'source': {'id': 'the-washington-post', 'name': 'The Washington Post'},
  'author': 'Mark Johnson',
  'title': 'Scientists develop gene delivery ‘trucks’ that could treat brain diseases - The Washington Post',
  'description': "Gene delivery systems can target specific brain cells, a breakthrough that could lead to treatments for Parkinson's, Alzheimer's and other neurodegenerative diseases.",
  'url': 'https://www.washingtonpost.com/science/2025/05/28/gene-delivery-brain-diseases/',
  'urlToImage': 'https://www.washingtonpost.com/wp-apps/imrs.php?src=https://arc-anglerfish-washpost-prod-washpost.s3.amazonaws.com/public/PUDZK5TT3H45HMKXNYCZVQOZ3Q_size-normalized.jpg&w=1440',
  'publishedAt': '2025-05-28T18:39:44Z',
  'content': 'Scores of researchers have produced new tools that can deliver genes and selectively activate them in hundreds of different cell types in the brain and spinal cord, a breakthrough that scientists hop… [+6646 chars]'},
 {'source': {'id': None, 'name': 'Verywel

In [6]:
df = pd.json_normalize(headlines['articles'])

In [7]:
df.head()

,author,title,description,url,urlToImage,publishedAt,content,source.id,source.name
0,Mark Johnson,Scientists develop gene delivery ‘trucks’ that...,Gene delivery systems can target specific brai...,https://www.washingtonpost.com/science/2025/05...,https://www.washingtonpost.com/wp-apps/imrs.ph...,2025-05-28T18:39:44Z,Scores of researchers have produced new tools ...,the-washington-post,The Washington Post
1,https://www.facebook.com/verywell,What Happens to Your Heart Health When You Eat...,"Sardines contain omega-3s, calcium, vitamin D,...",https://www.verywellhealth.com/are-sardines-go...,https://www.verywellhealth.com/thmb/OSPDiE6B9D...,2025-05-28T18:01:02Z,"Love or hate them, sardines are making waves i...",None,Verywellhealth.com
2,Carla K. Johnson,A new variant of COVID-19 may be driving up ca...,A new variant of COVID-19 is circulating in pa...,https://apnews.com/article/covid-variant-who-2...,https://dims.apnews.com/dims4/default/b494076/...,2025-05-28T16:28:00Z,COVID-19 cases are rising again as a new varia...,associated-press,Associated Press
3,Emily Weaver,Are Your Daily Naps Shortening Your Lifespan? ...,Are your naps putting your health at risk? Res...,https://bestlifeonline.com/adult-napping-morta...,https://bestlifeonline.com/media-library/image...,2025-05-28T15:26:42Z,If your naps tend to block out the entire afte...,None,Best Life
4,Beat of Hawaii,Guidance For Hawaii Visitors Following Zika Fo...,Hawaii just confirmed its first Zika case in y...,https://beatofhawaii.com/guidance-for-hawaii-v...,https://beatofhawaii.com/wp-content/uploads/20...,2025-05-28T14:26:49Z,Hawaii health officials have just confirmed a ...,None,Beatofhawaii.com


In [9]:
df = df.rename(columns={'publishedAt': 'published_date', 'source.name':'source_name'})

In [11]:
df.drop(columns=['urlToImage','source.id'], inplace = True)

In [12]:
df.columns

Index(['author', 'title', 'description', 'url', 'published_date', 'content',
       'source_name'],
      dtype='object')

In [ ]:

import psycopg2
db_params = {
    "dbname": "",
    "user": "",
    "password": '',
    "host": "",
    "port": ""
}


try:
    # Connect to PostgreSQL
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()
    
    # SQL Insert Query
    insert_query = """
    INSERT INTO health_news (author, title, description, url, published_date, content,source_name)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT (url) DO NOTHING;  -- Avoids duplicate primary key errors
    """
    
    # Insert DataFrame records one by one
    for _, row in df.iterrows():
        cursor.execute(insert_query, (
            row['author'], row['title'], row['description'],  row['url'], row['published_date'], row['content'],
            row['source_name']
        ))

    # Commit and close
    conn.commit()
    print("Data inserted successfully!")

except Exception as e:
    print("Error:", e)

finally:
    if conn:
        cursor.close()
        conn.close()

Data inserted successfully!
